# Convergence

In [ ]:
trickle =   {
                "1": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 0,
                    "Nreset": 1,
                    "counter": 1,
                    "epsilon": 1,
                    "k": 1,
                    "listen_period": 1048.0747845201658,
                    "m": 0,
                    "pfree": 0.33333333333333337,
                    "poccupancy": 0.6666666666666666,
                    "preset": 1.0,
                    "pstable": 0.0,
                    "state": 1
                },
                "2": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 1,
                    "Nreset": 1,
                    "counter": 1,
                    "epsilon": 0.802,
                    "k": 3,
                    "listen_period": 4476.3997287972625,
                    "m": 1,
                    "pfree": 0.6,
                    "poccupancy": 0.4,
                    "preset": 1.0,
                    "pstable": 0.0,
                    "state": 2
                },
                "3": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 2,
                    "Nreset": 1,
                    "counter": 1,
                    "epsilon": 0.6040000000000001,
                    "k": 4,
                    "listen_period": 5788.592057539519,
                    "m": 2,
                    "pfree": 0.7142857142857143,
                    "poccupancy": 0.2857142857142857,
                    "preset": 1.0,
                    "pstable": 0.0,
                    "state": 3
                },
                "4": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 3,
                    "Nreset": 1,
                    "counter": 1,
                    "epsilon": 0.4060000000000001,
                    "k": 4,
                    "listen_period": 1589.226230178494,
                    "m": 3,
                    "pfree": 0.7272727272727273,
                    "poccupancy": 0.2727272727272727,
                    "preset": 1.0,
                    "pstable": 0.0,
                    "state": 4
                },
                "5": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 4,
                    "Nreset": 2,
                    "counter": 0,
                    "epsilon": 0.20800000000000007,
                    "k": 3,
                    "listen_period": 1346.3197772448225,
                    "m": 0,
                    "pfree": 1.0,
                    "poccupancy": 0.0,
                    "preset": 0.4,
                    "pstable": 0.6,
                    "state": 5
                },
                "6": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 5,
                    "Nreset": 2,
                    "counter": 0,
                    "epsilon": 0.010000000000000064,
                    "k": 3,
                    "listen_period": 3898.039770082709,
                    "m": 1,
                    "pfree": 1.0,
                    "poccupancy": 0.0,
                    "preset": 0.4,
                    "pstable": 0.6,
                    "state": 6
                },
                "7": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 6,
                    "Nreset": 2,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 7506.93589907553,
                    "m": 2,
                    "pfree": 0.875,
                    "poccupancy": 0.125,
                    "preset": 0.4,
                    "pstable": 0.6,
                    "state": 7
                },
                "8": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 7,
                    "Nreset": 2,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 15183.222965729808,
                    "m": 3,
                    "pfree": 0.8823529411764706,
                    "poccupancy": 0.11764705882352941,
                    "preset": 0.4,
                    "pstable": 0.6,
                    "state": 8
                },
                "9": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 8,
                    "Nreset": 3,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 1232.8669891243894,
                    "m": 0,
                    "pfree": 0.6666666666666667,
                    "poccupancy": 0.3333333333333333,
                    "preset": 0.3333333333333333,
                    "pstable": 0.6666666666666667,
                    "state": 9
                },
                "10": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 9,
                    "Nreset": 3,
                    "counter": 1,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 1154.4759478424048,
                    "m": 1,
                    "pfree": 0.8333333333333334,
                    "poccupancy": 0.16666666666666666,
                    "preset": 0.3333333333333333,
                    "pstable": 0.6666666666666667,
                    "state": 10
                },
                "11": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 10,
                    "Nreset": 3,
                    "counter": 1,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 1286.7463902388645,
                    "m": 2,
                    "pfree": 0.8,
                    "poccupancy": 0.2,
                    "preset": 0.3333333333333333,
                    "pstable": 0.6666666666666667,
                    "state": 11
                },
                "12": {
                    "DIOsurpress": 1,
                    "DIOtransmit": 11,
                    "Nreset": 3,
                    "counter": 3,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 267.7176776616561,
                    "m": 3,
                    "pfree": 0.7894736842105263,
                    "poccupancy": 0.21052631578947367,
                    "preset": 0.3333333333333333,
                    "pstable": 0.6666666666666667,
                    "state": 12
                },
                "13": {
                    "DIOsurpress": 2,
                    "DIOtransmit": 11,
                    "Nreset": 3,
                    "counter": 1,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 29714.897584848957,
                    "m": 4,
                    "pfree": 0.6578947368421053,
                    "poccupancy": 0.34210526315789475,
                    "preset": 0.3333333333333333,
                    "pstable": 0.6666666666666667,
                    "state": 13
                },
                "14": {
                    "DIOsurpress": 3,
                    "DIOtransmit": 11,
                    "Nreset": 3,
                    "counter": 4,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 75460.45734896042,
                    "m": 5,
                    "pfree": 0.8181818181818181,
                    "poccupancy": 0.18181818181818182,
                    "preset": 0.3333333333333333,
                    "pstable": 0.6666666666666667,
                    "state": 14
                },
                "15": {
                    "DIOsurpress": 3,
                    "DIOtransmit": 12,
                    "Nreset": 4,
                    "counter": 1,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 2672.644958188831,
                    "m": 0,
                    "pfree": 0.6666666666666667,
                    "poccupancy": 0.3333333333333333,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 15
                },
                "16": {
                    "DIOsurpress": 3,
                    "DIOtransmit": 13,
                    "Nreset": 4,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 2209.8995324345296,
                    "m": 1,
                    "pfree": 1.0,
                    "poccupancy": 0.0,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 16
                },
                "17": {
                    "DIOsurpress": 3,
                    "DIOtransmit": 14,
                    "Nreset": 4,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 3955.0309459656382,
                    "m": 2,
                    "pfree": 0.7,
                    "poccupancy": 0.3,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 17
                },
                "18": {
                    "DIOsurpress": 3,
                    "DIOtransmit": 15,
                    "Nreset": 4,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 2184.37014849118,
                    "m": 3,
                    "pfree": 0.9565217391304348,
                    "poccupancy": 0.043478260869565216,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 18
                },
                "19": {
                    "DIOsurpress": 4,
                    "DIOtransmit": 15,
                    "Nreset": 4,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 21189.297507340336,
                    "m": 4,
                    "pfree": 0.8947368421052632,
                    "poccupancy": 0.10526315789473684,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 19
                },
                "20": {
                    "DIOsurpress": 5,
                    "DIOtransmit": 15,
                    "Nreset": 4,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 40636.352227019335,
                    "m": 5,
                    "pfree": 0.8658536585365854,
                    "poccupancy": 0.13414634146341464,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 20
                },
                "21": {
                    "DIOsurpress": 6,
                    "DIOtransmit": 15,
                    "Nreset": 4,
                    "counter": 3,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 157698.4478691334,
                    "m": 6,
                    "pfree": 0.8313953488372093,
                    "poccupancy": 0.1686046511627907,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 21
                },
                "22": {
                    "DIOsurpress": 7,
                    "DIOtransmit": 15,
                    "Nreset": 4,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 4,
                    "listen_period": 312483.3430377628,
                    "m": 7,
                    "pfree": 0.8397790055248618,
                    "poccupancy": 0.16022099447513813,
                    "preset": 0.26666666666666666,
                    "pstable": 0.7333333333333334,
                    "state": 22
                },
                "23": {
                    "DIOsurpress": 7,
                    "DIOtransmit": 16,
                    "Nreset": 5,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 1022.1638155571102,
                    "m": 0,
                    "pfree": 1.0,
                    "poccupancy": 0.0,
                    "preset": 0.21739130434782608,
                    "pstable": 0.782608695652174,
                    "state": 23
                },
                "24": {
                    "DIOsurpress": 7,
                    "DIOtransmit": 17,
                    "Nreset": 5,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 1440.926936994931,
                    "m": 1,
                    "pfree": 0.6666666666666667,
                    "poccupancy": 0.3333333333333333,
                    "preset": 0.21739130434782608,
                    "pstable": 0.782608695652174,
                    "state": 24
                },
                "25": {
                    "DIOsurpress": 7,
                    "DIOtransmit": 18,
                    "Nreset": 5,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 4133.887974670645,
                    "m": 2,
                    "pfree": 0.7692307692307692,
                    "poccupancy": 0.23076923076923078,
                    "preset": 0.21739130434782608,
                    "pstable": 0.782608695652174,
                    "state": 25
                },
                "26": {
                    "DIOsurpress": 7,
                    "DIOtransmit": 19,
                    "Nreset": 5,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 9997.444472908699,
                    "m": 3,
                    "pfree": 0.88,
                    "poccupancy": 0.12,
                    "preset": 0.21739130434782608,
                    "pstable": 0.782608695652174,
                    "state": 26
                },
                "27": {
                    "DIOsurpress": 8,
                    "DIOtransmit": 19,
                    "Nreset": 5,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 28315.51198332584,
                    "m": 4,
                    "pfree": 0.8043478260869565,
                    "poccupancy": 0.1956521739130435,
                    "preset": 0.21739130434782608,
                    "pstable": 0.782608695652174,
                    "state": 27
                },
                "28": {
                    "DIOsurpress": 9,
                    "DIOtransmit": 19,
                    "Nreset": 5,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 27915.85492267603,
                    "m": 5,
                    "pfree": 0.8144329896907216,
                    "poccupancy": 0.18556701030927836,
                    "preset": 0.21739130434782608,
                    "pstable": 0.782608695652174,
                    "state": 28
                },
                "29": {
                    "DIOsurpress": 9,
                    "DIOtransmit": 20,
                    "Nreset": 6,
                    "counter": 1,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 933.8165355334552,
                    "m": 0,
                    "pfree": 0.5,
                    "poccupancy": 0.5,
                    "preset": 0.20689655172413793,
                    "pstable": 0.7931034482758621,
                    "state": 29
                },
                "30": {
                    "DIOsurpress": 9,
                    "DIOtransmit": 21,
                    "Nreset": 6,
                    "counter": 1,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 4254.7385259127595,
                    "m": 1,
                    "pfree": 0.75,
                    "poccupancy": 0.25,
                    "preset": 0.20689655172413793,
                    "pstable": 0.7931034482758621,
                    "state": 30
                },
                "31": {
                    "DIOsurpress": 9,
                    "DIOtransmit": 22,
                    "Nreset": 6,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 351.4814424154674,
                    "m": 2,
                    "pfree": 0.8461538461538461,
                    "poccupancy": 0.15384615384615385,
                    "preset": 0.20689655172413793,
                    "pstable": 0.7931034482758621,
                    "state": 31
                },
                "32": {
                    "DIOsurpress": 9,
                    "DIOtransmit": 23,
                    "Nreset": 6,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 11799.181871954652,
                    "m": 3,
                    "pfree": 0.7916666666666666,
                    "poccupancy": 0.20833333333333334,
                    "preset": 0.20689655172413793,
                    "pstable": 0.7931034482758621,
                    "state": 32
                },
                "33": {
                    "DIOsurpress": 10,
                    "DIOtransmit": 23,
                    "Nreset": 6,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 46202.71874000181,
                    "m": 4,
                    "pfree": 0.8571428571428572,
                    "poccupancy": 0.14285714285714285,
                    "preset": 0.20689655172413793,
                    "pstable": 0.7931034482758621,
                    "state": 33
                },
                "34": {
                    "DIOsurpress": 11,
                    "DIOtransmit": 23,
                    "Nreset": 6,
                    "counter": 0,
                    "epsilon": 0.01,
                    "k": 3,
                    "listen_period": 37287.12274417339,
                    "m": 5,
                    "pfree": 0.8210526315789474,
                    "poccupancy": 0.17894736842105263,
                    "preset": 0.20689655172413793,
                    "pstable": 0.7931034482758621,
                    "state": 34
                }
            }

In [ ]:
import pandas as pd


df_ = pd.DataFrame(columns=trickle["1"].keys())

for key in trickle.keys():
    val = trickle[key]
    val["state"] = key
    df_ = df_.append(val, ignore_index = True)

df_.to_csv("convergence.csv", index=False)

# Grouped bar

In [ ]:
import pandas as pd 

df = pd.read_csv("simulator/bin/simData/comparison_merged.csv")
df

In [ ]:
df_ = df.copy()
df_

In [ ]:
df_t = df_.pivot(index='parameter', columns='method', values=df_.columns[2:].to_list())
df_t = df_t.reset_index()
df_t.columns = df_t.columns.map('-'.join)
df_t

In [ ]:
df_t.to_csv('dio-transmission.csv', index=False)